In [11]:
import numpy as np
import pygame
import math

np.random.seed(42)

# Constants
Nx, Ny = 50, 50
dx=min(500/Nx, 500/Ny)
du, dv = 0.03, 0.08
au, av, bu, bv = 0.08, 0.6, -0.07, 0.0
cu, cv, Du, Dv = 0.03, -0.015, 0.02, 1.5
Fmax, Gmax = 0.2, 0.5
dt = 0.1

# Initialize grid
u = np.random.rand(Nx * Ny)
v = np.random.rand(Nx * Ny)

# Offset coordinate neighbors for even-q vertical hex layout
def get_hex_neighbors(Nx, Ny):
    neighbors = [[] for _ in range(Nx * Ny)]
    for x in range(Nx):
        for y in range(Ny):
            idx = x + y * Nx
            odd = y % 2 == 1
            if odd:
                directions = [(0, -2), (0, 2), (0, -1), (1, -1), (0, 1), (1, 1)]
            else:
                directions = [(0, -2), (0, 2), (-1, -1), (0, -1), (-1, 1), (0, 1)]


            for dx_, dy_ in directions:
                nx = (x + dx_) % Nx
                ny = (y + dy_) % Ny
                nidx = nx + ny * Nx
                neighbors[idx].append(nidx)
    return neighbors

neighbors = get_hex_neighbors(Nx, Ny)

# Evolution step
def evolucion(u, v):
    aux_u = u.copy()
    aux_v = v.copy()

    for n in range(Nx * Ny):
        F = max(-Fmax, min((au - du) * u[n] + bu * v[n] + cu, Fmax))
        G = max(-Gmax, min(av * u[n] + (bv - dv) * v[n] + cv, Gmax))

        u_sum = sum(u[nb] for nb in neighbors[n])
        v_sum = sum(v[nb] for nb in neighbors[n])

        aux_u[n] += dt * (F + Du * (u_sum - 6 * u[n]))
        aux_v[n] += dt * (G + Dv * (v_sum - 6 * v[n]))

    u[:] = np.clip(aux_u, 0, 1)
    v[:] = np.clip(aux_v, 0, 1)

# Pygame visualization function
def draw_hex(surface, color, x, y, size):
    points = []
    for i in range(6):
        angle = math.pi / 3 * i
        px = x + size * math.cos(angle)
        py = y + size * math.sin(angle)
        points.append((px, py))
    pygame.draw.polygon(surface, color, points)

def visualize(u, t):
    screen.fill((0, 0, 0))

    minimo_u = np.min(u)
    maximo_u = np.max(u)
    hex_height = math.sqrt(3) * dx
    hex_width = dx * 2
    horiz = hex_width + dx
    vert = hex_height / 2

    for x in range(Nx):
        for y in range(Ny):
            idx = x + y * Nx
            u_val = u[idx]
            u_norm = (u_val - minimo_u) / (maximo_u - minimo_u + 1e-8)

            color = (int(u_norm * 255), int(u_norm * 255), int(u_norm * 255))

            x_pos = (x + 0.5) * horiz + (horiz / 2 if y % 2 else 0)
            y_pos = (y + 1) * vert

            draw_hex(screen, color, x_pos, y_pos, hex_width*0.5)

    # Time display
    time_text = font.render(f"Time: {t * dt:.2f}", True, (255, 255, 255))
    screen.blit(time_text, (10, (Ny-1) * dx ))

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((int((Nx + 1) * dx * 5/2), int((Ny + 1) * dx)))
pygame.display.set_caption("Hexagonal Grid Simulation")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 24)

# Main loop
running = True
t = 0
visualize(u, t)

while running and t < 5000000:
    t += 1
    evolucion(u, v)

    if t % 10 == 0:
        visualize(u, t)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()
    clock.tick(60)

pygame.image.save(screen, "pattern_hexagonal.png")
print("Saved final state as pattern_hexagonal.png")

pygame.quit()


Saved final state as pattern_hexagonal.png


In [33]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import math

# Grid size
Nx, Ny = 25, 100
dx = 1.0
hex_height = math.sqrt(3) * dx
hex_width = dx * 2
horiz = hex_width + dx
vert = hex_height / 2

# Load data
u = np.loadtxt("hex_result.txt")

# Normalize to [0, 1]
min_u = np.min(u)
max_u = np.max(u)
u_norm = (u - min_u) / (max_u - min_u + 1e-8)

# Generate hexagon centers and vertices
verts = []
colors = []

for x in range(Nx):
    for y in range(Ny):
        idx = x + y * Nx
        val = u_norm[idx]
        gray = val  # black = high, white = low

        # Compute hex center
        x_pos = (x + 0.5) * horiz + (horiz / 2 if y % 2 else 0)
        y_pos = (y + 1) * vert

        # Create hexagon vertices
        hexagon = []
        for i in range(6):
            angle = math.pi / 3 * i
            px = x_pos + dx * math.cos(angle)
            py = y_pos + dx * math.sin(angle)
            hexagon.append((px, py))

        verts.append(hexagon)
        colors.append((gray, gray, gray))  # RGB triplet for grayscale

# Plot using PolyCollection
fig, ax = plt.subplots(figsize=(12, 12), dpi=300)
collection = PolyCollection(verts, facecolors=colors, edgecolors=colors, linewidths=0)
ax.add_collection(collection)

ax.set_aspect('equal')
ax.autoscale_view()
ax.axis('off')

# Save image
plt.tight_layout()
plt.savefig("hex_grid_visualization.png", dpi=600, bbox_inches='tight')
plt.close()

print("✅ Image saved as hex_final_grayscale.png")


✅ Image saved as hex_final_grayscale.png
